This notebook documents how to solve the famous [zebra puzzle](http://en.wikipedia.org/wiki/Zebra_puzzle) by A. Einstein with the help of semantic web tools for python.


It is based on theses Links/Projects

- https://pythonhosted.org/Owlready2/index.html
- https://github.com/RDFLib/OWL-RL/issues/3 "Solving Einstein's riddle (zebra puzzle"
    - Linked owl-files:
    - https://github.com/RDFLib/OWL-RL/files/1533408/einsteins_riddle.owl.txt ← works
    - https://github.com/RDFLib/OWL-RL/files/1533409/zebra.n3.txt  ← works not yet
- http://lambdamusic.github.io/Ontospy/


For installation I did:

```bash
pip install owlready
pip install ontospy[FULL]
``` 


In [1]:
import os
import owlready2 as owl2

# Direct approach

we assume that both ontology files have been downloaded to `./ontology_data/<filename>`. They are parsed directly with owlready.

## Ontology 1

In [2]:
data_path = "ontology_data"
path1 = os.path.join(data_path, "einsteins_riddle.owl.txt")
# path1 = os.path.join(data_path, "einsteins_riddle_manchester.owl")


In [3]:
onto = owl2.get_ontology(path1).load()
print("classes (concepts):\n", list(onto.classes()), "\n"*2)
print("properties (roles):\n", list(onto.properties()), "\n"*2)
print("individuals:\n", list(onto.individuals()))

classes (concepts):
 [owl.Nothing, einsteins_riddle.owl.txt.Beverage, einsteins_riddle.owl.txt.Man, einsteins_riddle.owl.txt.Color, einsteins_riddle.owl.txt.House, einsteins_riddle.owl.txt.Pet, einsteins_riddle.owl.txt.Cigarette] 


properties (roles):
 [einsteins_riddle.owl.txt.drinks, einsteins_riddle.owl.txt.has_color, einsteins_riddle.owl.txt.lives_in, einsteins_riddle.owl.txt.owns, einsteins_riddle.owl.txt.right_to, einsteins_riddle.owl.txt.smokes, 1.1.description, 1.1.date, 1.1.creator] 


individuals:
 [einsteins_riddle.owl.txt.orange_juice, einsteins_riddle.owl.txt.tea, einsteins_riddle.owl.txt.water, einsteins_riddle.owl.txt.coffee, einsteins_riddle.owl.txt.milk, einsteins_riddle.owl.txt.Kools, einsteins_riddle.owl.txt.Lucky_Strike, einsteins_riddle.owl.txt.Parliaments, einsteins_riddle.owl.txt.Chesterfields, einsteins_riddle.owl.txt.Old_Gold, einsteins_riddle.owl.txt.green, einsteins_riddle.owl.txt.red, einsteins_riddle.owl.txt.blue, einsteins_riddle.owl.txt.yellow, einsteins

In [4]:
# onto.save("test.owl", format="ntriples")

### Relations before calling the reasoner

In [5]:
list(onto.lives_in.get_relations())

[(einsteins_riddle.owl.txt.Norwegian, einsteins_riddle.owl.txt.house_1)]

In [6]:
list(onto.owns.get_relations())

[(einsteins_riddle.owl.txt.Spaniard, einsteins_riddle.owl.txt.dog)]

In [7]:
# tmp !!!

In [8]:
onto.Englishman.lives_in is None

True

In [9]:
axioms = list(onto.general_axioms())

def print_ax(ax, i=0):
    print(i, str(ax).replace("einsteins_riddle.owl.txt", ":"), "\n")
        

In [10]:
print_ax(axioms[0])
print_ax(axioms[1])
print_ax(axioms[2])

0 :.lives_in.value(:.house_3) 

0 Inverse(:.drinks).some(:.lives_in.value(:.house_3)) 

0 Inverse(:.right_to).only(owl.Nothing) 



In [11]:
onto.lives_in.value(onto.house_3) == axioms[0]

True

In [12]:
onto.Englishman.equivalent_to.append(axioms[0])
onto.Englishman.equivalent_to

[einsteins_riddle.owl.txt.lives_in.value(einsteins_riddle.owl.txt.house_3)]

In [16]:
onto.Englishman.equivalent_to

[einsteins_riddle.owl.txt.lives_in.value(einsteins_riddle.owl.txt.house_3)]

In [13]:
1/0

ZeroDivisionError: division by zero

In [ ]:
for i, ax in enumerate(axioms[:4]):
    print_ax(ax, i)

In [ ]:
type(axioms[0])

In [ ]:
list(onto.world.classes())

### Calling the reasoner and inspect relations again

In [ ]:
%time owl2.sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True, debug=0)

In [ ]:
list(onto.lives_in.get_relations())

In [ ]:
list(onto.owns.get_relations())

In [ ]:
list(onto.lives_in.get_relations())

In [ ]:
h3 = list(onto.individuals())[16]
h3.name

In [ ]:
h3.right_to

In [ ]:
onto.Pet.is_a

In [ ]:
onto.Man.is_a

In [ ]:
onto.Pet.equivalent_to

### Conclusion

The puzzle could be solved easily by the reasoner